## DUSP1 Dataframe Concatenation and Replica Check
- Concatenate experimental dataframes based on specific conditions:
    - **100nM 3hr Time-Sweep (TS):** Data collected over a 3-hour time period with a fixed concentration of 100nM.
    - **75min Concentration-Sweep:** Data collected over varying concentrations during a 75-minute time period.
    - **3hr Time-Concentration Sweep (TCS):** Data collected over a 3-hour period with varying concentrations.
    - **Triptiolide (TPL):** Data collected under conditions involving Triptiolide treatment.
- Perform a replica check to ensure data consistency and identify any discrepancies across experimental replicates.
- Document and visualize the concatenated data for further analysis.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import sys
import datetime

import h5py
import dask.array as da

# src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# print(src_path)
# sys.path.append(src_path)

# from src.Analysis_DUSP1_v2 import DUSP1DisplayManager

## DUSP1 TS Experiments
- Replica's D, E, F
- Partial's M, N

In [3]:
# Load in the dataframes

DUSP1_RepD_results = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_D_cell_level_results_Apr21_filtered_MG3.csv')
DUSP1_RepD_spots = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_D_merged_spots_Apr21_filtered_MG3.csv')
DUSP1_RepD_clusters = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_D_merged_clusters_Apr21_filtered_MG3.csv')

DUSP1_RepE_results = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_E_cell_level_results_Apr21_filtered_MG3.csv')
DUSP1_RepE_spots = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_E_merged_spots_Apr21_filtered_MG3.csv')
DUSP1_RepE_clusters = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_E_merged_clusters_Apr21_filtered_MG3.csv')

DUSP1_RepF_results = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_F_cell_level_results_Apr21_filtered_MG3.csv')
DUSP1_RepF_spots = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_F_merged_spots_Apr21_filtered_MG3.csv')
DUSP1_RepF_clusters = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_F_merged_clusters_Apr21_filtered_MG3.csv')

DUSP1_RepM_results = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_M_cell_level_results_Apr21_filtered_MG3.csv')
DUSP1_RepM_spots = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_M_merged_spots_Apr21_filtered_MG3.csv')
DUSP1_RepM_clusters = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_M_merged_clusters_Apr21_filtered_MG3.csv')

DUSP1_RepN_results = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_N_cell_level_results_Apr21_filtered_MG3.csv')
DUSP1_RepN_spots = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_N_merged_spots_Apr21_filtered_MG3.csv')
DUSP1_RepN_clusters = pd.read_csv('/Users/ericron/Desktop/AngelFISH/Publications/Ron_2024/dataframes/DUSP1_N_merged_clusters_Apr21_filtered_MG3.csv')

print(DUSP1_RepD_results.keys())

Index(['Unnamed: 0', 'unique_cell_id', 'touching_border', 'MG_count',
       'nuc_MG_count', 'cyto_MG_count', 'num_ts', 'num_foci', 'num_spots_foci',
       'num_spots_ts', 'largest_ts', 'second_largest_ts', 'nuc_area_px',
       'cyto_area_px', 'avg_nuc_int', 'avg_cyto_int', 'avg_cell_int',
       'std_cell_int', 'time', 'dex_conc', 'replica', 'fov', 'nas_location',
       'h5_idx'],
      dtype='object')


In [ ]:
# Concatenate the results dataframes 